# patient

The patinet table is a core part of the eICU-CRD and contains all information related to tracking patient unit stays. The table also contains patient demographics and hospital level information.

In [1]:
# Import libraries
import pandas as pd
# import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from pandas.io import gbq
import tensorflow as tf

%matplotlib inline

In [2]:
project_id='datathon-tarragona-2018'

# Read data from BigQuery into pandas dataframes.
def run_query(query):
  return pd.io.gbq.read_gbq(query, project_id=project_id, configuration={'query':{'useLegacySql': False}})

## uniquePid

The `uniquePid` column identifies a single patient across multiple stays. Let's look at a single `uniquepid`.

In [3]:
uniquepid = '002-33870'
df = run_query("""select *
from `physionet-data.eicu_crd.patient` 
where uniquepid = '{0}'""".format(uniquepid))

df.head()

,patientunitstayid,patienthealthsystemstayid,gender,age,ethnicity,hospitalid,wardid,apacheadmissiondx,admissionheight,hospitaladmittime24,...,unitadmitsource,unitvisitnumber,unitstaytype,admissionweight,dischargeweight,unitdischargetime24,unitdischargeoffset,unitdischargelocation,unitdischargestatus,uniquepid
0,141179,128927,Female,52,Caucasian,60,83,,162.6,08:56:00,...,ICU to SDU,2,stepdown/other,NaN,60.4,19:20:00,2042,Home,Alive,002-33870
1,141178,128927,Female,52,Caucasian,60,83,,162.6,08:56:00,...,Emergency Department,1,admit,54.4,54.4,09:18:00,8,Step-Down Unit (SDU),Alive,002-33870


Here we see two unit stays for a single patient. Note also that both unit stays have the same `patienthealthsystemstayid` - this indicates that they occurred within the same hospitalization.

We can see the `unitstaytype` was 'admit' for one stay, and 'stepdown/other' for another. Other columns can give us more information.

In [4]:
df[['patientunitstayid', 'wardid', 'unittype', 'unitstaytype', 'hospitaladmitoffset', 'unitdischargeoffset']]

,patientunitstayid,wardid,unittype,unitstaytype,hospitaladmitoffset,unitdischargeoffset
0,141179,83,Med-Surg ICU,stepdown/other,-22,2042
1,141178,83,Med-Surg ICU,admit,-14,8


Note that it's not explicitly obvious which stay occurred first. Earlier stays will be closer to hospital admission, and therefore have a *higher* hospitaladmitoffset. Above, the stay with a `hospitaladmitoffset` of -14 was first (occurring 14 minutes after hospital admission), followed by the next stay with a `hospitaladmitoffset` of 22 (which occurred 22 minutes after hospital admission). Practically, we wouldn't consider the first admission a "real" ICU stay, and it's likely an idiosyncrasy of the administration system at this particular hospital. Notice how both rows have the same `wardid`.

## Age

As ages over 89 are required to be deidentified by HIPAA, the `age` column is actually a string field, with ages over 89 replaced with the string value '> 89'.

In [6]:
df = run_query("""select age, count(*) as n
from `physionet-data.eicu_crd.patient` 
group by age
order by n desc""")

df.head()

,age,n
0,> 89,7081
1,67,5078
2,68,4826
3,72,4804
4,71,4764


As is common in eICU-CRD, there are a subset of hospitals who routinely utilize this portion of the medical record (and thus have 90-100% data completion), while there are other hospitals who rarely use this interface and thus have poor data completion (0-10%).